In [10]:
cv = None
count_matrix = None
cosine_sim = None
indices = None

Matriz de recuento

In [17]:
import pandas as pd
import joblib

# Cargar el archivo model.pkl que contiene las columnas 'genres', 'tags' y 'specs'
model_data = joblib.load('model.pkl')

# Crear un DataFrame a partir de los datos cargados
df_games = pd.DataFrame(model_data)

# Ahora df_games contiene las columnas 'genres', 'tags' y 'specs' desde model.pkl




In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Lista de columnas relevantes
columnas_similares = ['genres', 'tags', 'specs']

# Función para combinar listas en una cadena
def combinar_listas(row):
    combined = ' '.join(item for sublist in row if sublist for item in sublist)
    return combined

# Crear una nueva columna "variables_juntas" con las listas combinadas en cadenas
df_games['variables_juntas'] = df_games[columnas_similares].apply(combinar_listas, axis=1)

# Crear un vectorizador TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Calcular la matriz TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform(df_games['variables_juntas'])





ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
@app.on_event("startup")
async def load_data():
    global df,df_highly_rated, cv, count_matrix, nn, indices

    # Carga los datos
    df_highly_rated = pd.read_csv('movies_final_with_combined_features2.csv')
    df = pd.read_csv('movies_final.csv')

    # Computa la matriz de recuento
    cv = CountVectorizer(stop_words='english', max_features=5000)
    count_matrix = cv.fit_transform(df_highly_rated['combined_features'])

    # Crea un modelo NearestNeighbors
    nn = NearestNeighbors(metric='cosine', algorithm='brute')
    nn.fit(count_matrix)

    # Crea un índice de títulos de películas
    indices = pd.Series(df_highly_rated.index, index=df_highly_rated['title']).drop_duplicates()


In [14]:
import pandas as pd
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer

# Cargar el archivo model.pkl que contiene las columnas 'genres', 'tags' y 'specs'
model_data = joblib.load('model.pkl')

# Crear un DataFrame a partir de los datos cargados
df_games = pd.DataFrame(model_data)

# Lista de columnas relevantes
columnas_similares = ['genres', 'tags', 'specs']

# Función para combinar listas en una cadena
def combinar_listas(row):
    combined = ' '.join(item for sublist in row if sublist for item in sublist)
    return combined

# Crear una nueva columna "variables_juntas" con las listas combinadas en cadenas
df_games['variables_juntas'] = df_games[columnas_similares].apply(combinar_listas, axis=1)

# Crear una matriz TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_games['variables_juntas'])

# Ahora, tfidf_matrix contiene la matriz TF-IDF de las columnas 'genres', 'tags' y 'specs' en df_games.


ValueError: empty vocabulary; perhaps the documents only contain stop words

In [ ]:
@app.get('/recomendacion/{title}')
def recomendacion(title):
    '''Ingresas un nombre de pelicula y te recomienda las similares en una lista
    ''' 
    # Verifica si el título ingresado se encuentra en el df
    if title not in df_highly_rated['title'].values:
        return 'La pelicula no se encuentra en el conjunto de la base de datos.'
    else:
        # Si el título está en el df, encuentra su índice
        index = indices[title]

        # Obtiene las puntuaciones de similitud de las 5 películas más cercanas
        distances, indices_knn = nn.kneighbors(count_matrix[index], n_neighbors=6)  # n_neighbors=6 because the first match will be the movie itself

        # Obtiene los índices de las películas
        movie_indices = indices_knn[0][1:]  # Skip the first match (the movie itself)

        # Devuelve las 5 peliculas mas similares
        return df_highly_rated['title'].iloc[movie_indices].tolist()